## References

##### Bureau of Transportation Statistics

https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236  

https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

##### Additional sources
This might be useful in matching station codes to airports:

http://dss.ucar.edu/datasets/ds353.4/inventories/station-list.html

https://www.world-airport-codes.com/

## Libraries

In [0]:
import re
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import os
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import OneHotEncoder

from pyspark.ml.classification import LogisticRegression

sqlContext = SQLContext(sc)

In [0]:
# helper function
# Missing Values
# count the number of null values per column
def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([f.count(f.when(f.isnull(c) | f.isnan(c), c)).alias(c) for c in spark_df.columns]).toPandas()
    if len(df) == 0:
        print("There are no any missing values!")
        return None
    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)
    return df


## Import Data

##### Import

In [0]:
# airlines
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/201*.parquet")

# weather
weather = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather201*a.parquet")

# stations
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/DEMO8/gsod/stations.csv.gz")

# codes to join weather station and airports
codes = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/agao729@berkeley.edu/airports.csv")

##### Shape

In [0]:
print("airlines Rows, Columns: ", airlines.count(), "," ,len(airlines.columns))
print("weather Rows, Columns: ", weather.count(), "," ,len(weather.columns))
print("stations Rows, Columns: ", stations.count(), "," ,len(stations.columns))
print("codes Rows, Columns: ", codes.count(), "," ,len(codes.columns))

airlines Rows, Columns: 31746841 , 109
weather Rows, Columns: 626994336 , 177
stations Rows, Columns: 29771 , 11
codes Rows, Columns: 7698 , 14

## Airlines

### Bureau of Transportation Statistics
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

In [0]:
#df with missing values
missing_df = count_missings(airlines, sort=True)

In [0]:
# grab the columns that have more than 1/3 missing values from the original dataset
threshold = float(airlines.count())/3
missing_cols = missing_df[missing_df['count'] > threshold].index.values
missing_cols

Out[11]: array(['DIV5_TAIL_NUM', 'DIV4_TAIL_NUM', 'DIV4_AIRPORT',
 'DIV4_AIRPORT_ID', 'DIV4_AIRPORT_SEQ_ID', 'DIV4_WHEELS_ON',
 'DIV4_TOTAL_GTIME', 'DIV4_LONGEST_GTIME', 'DIV4_WHEELS_OFF',
 'DIV5_AIRPORT', 'DIV5_AIRPORT_ID', 'DIV5_AIRPORT_SEQ_ID',
 'DIV5_WHEELS_ON', 'DIV5_TOTAL_GTIME', 'DIV5_LONGEST_GTIME',
 'DIV5_WHEELS_OFF', 'DIV3_TAIL_NUM', 'DIV3_WHEELS_OFF',
 'DIV3_AIRPORT', 'DIV3_WHEELS_ON', 'DIV3_AIRPORT_ID',
 'DIV3_AIRPORT_SEQ_ID', 'DIV3_TOTAL_GTIME', 'DIV3_LONGEST_GTIME',
 'DIV2_TAIL_NUM', 'DIV2_WHEELS_OFF', 'DIV2_AIRPORT',
 'DIV2_LONGEST_GTIME', 'DIV2_TOTAL_GTIME', 'DIV2_WHEELS_ON',
 'DIV2_AIRPORT_SEQ_ID', 'DIV2_AIRPORT_ID',
 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV1_WHEELS_OFF',
 'DIV1_TAIL_NUM', 'DIV_DISTANCE', 'DIV_REACHED_DEST',
 'DIV1_WHEELS_ON', 'DIV1_AIRPORT', 'DIV1_TOTAL_GTIME',
 'DIV1_LONGEST_GTIME', 'DIV1_AIRPORT_ID', 'DIV1_AIRPORT_SEQ_ID',
 'LONGEST_ADD_GTIME', 'TOTAL_ADD_GTIME', 'FIRST_DEP_TIME',
 'CANCELLATION_CODE', 'NAS_DELAY', 'CARRIER_DELAY',
 'LATE_AIRCRAFT_DELAY', 'SECURITY_DELAY', 'WEATHER_DELAY'],
 dtype=object)

In [0]:
#drop missing values
airlines = airlines.drop(*missing_cols)
len(airlines.columns)
print("airlines Rows, Columns: ", airlines.count(), "," ,len(airlines.columns))

airlines Rows, Columns: 31746841 , 56

In [0]:
# Select columns that are not completely empty
irrev_cols = ['QUARTER', 'DAY_OF_MONTH']

redun_cols = ["OP_UNIQUE_CARRIER", "OP_CARRIER_AIRLINE_ID" , "OP_CARRIER_FL_NUM", "ORIGIN_AIRPORT_SEQ_ID", "ORIGIN_CITY_MARKET_ID", "ORIGIN_STATE_FIPS", "ORIGIN_STATE_NM", "ORIGIN_WAC", "DEST_AIRPORT_SEQ_ID", "DEST_CITY_MARKET_ID", "DEST_STATE_FIPS", "DEST_STATE_NM", "DEST_WAC", "DEP_DELAY", "DEP_DELAY_NEW", "ARR_DELAY", "ARR_DELAY_GROUP", "ARR_TIME_BLK", "DISTANCE_GROUP", "FIRST_DEP_TIME", "TOTAL_ADD_GTIME", "LONGEST_ADD_GTIME", "DIV_AIRPORT_LANDINGS", "DISTANCE_GROUP", "CANCELLED", "ACTUAL_ELAPSED_TIME", "AIR_TIME", "FLIGHTS", "DISTANCE_GROUP", "ARR_TIME", "ARR_DELAY_NEW", "ARR_DEL15", "ORIGIN_AIRPORT_ID","DEST_AIRPORT_ID","DEP_DELAY_GROUP", "ORIGIN_STATE_ABR", "DEST_STATE_ABR", "CANCELLATION_CODE", "TAIL_NUM"]

airlines = airlines.drop(*irrev_cols)
airlines = airlines.drop(*redun_cols)
print("airlines Rows, Columns: ", airlines.count(), "," ,len(airlines.columns))
airlines.columns

airlines Rows, Columns: 31746841 , 21
Out[13]: ['YEAR',
 'MONTH',
 'DAY_OF_WEEK',
 'FL_DATE',
 'OP_CARRIER',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'DEST',
 'DEST_CITY_NAME',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DEL15',
 'DEP_TIME_BLK',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'DISTANCE']

In [0]:
print("airlines Rows, Columns: ", airlines.count(), "," ,len(airlines.columns))

airlines Rows, Columns: 31746841 , 21

## Weather

We will mainly use data from the Mandatory Data Section and Additional Data Section:

__Mandatory Data Section__ - The mandatory data section contains meteorological
information on the basic elements such as winds, visibility, and temperature.
These are the most commonly reported parameters and are available most of the
time. The mandatory data section is fixed length and is 45 characters long.

__Additional Data Section__ - Variable length data are provided after the
mandatory data. These additional data contain information of significance
and/or which are received with varying degrees of frequency. Identifiers are
used to note when data are present in the record. If all data fields in a
group are missing, the entire group is usually not reported. If no groups are
reported the section will be omitted. The additional data section is variable
in length with a minimum of 0 characters and a maximum of 637 (634 characters
plus a 3 character section identifier) characters.

Missing Values are identified as :

__Missing Values__ - Missing values for any non-signed item are filled (i.e.,
999). Missing values for any signed item are positive filled (i.e., +99999).


Source: https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf

In [0]:
#Keep US records only and weather records for FM15 report type (Aviation routine weather report)
keep_weather_cols = ["STATION", "DATE", "LATITUDE", 'LONGITUDE', 'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'WND', 'VIS', 'TMP', 'DEW', 'SLP', 'AA1', 'AJ1', 'AT1', 'GA1', 'IA1', 'MA1']

weather = weather.withColumn("COUNTRY", f.substring(f.col("NAME"), -2, 2)).filter("COUNTRY = 'US'").filter("REPORT_TYPE LIKE '%FM-15%'").select(keep_weather_cols)
print("Rows, Columns: ", weather.count(), "," ,len(weather.columns))

Rows, Columns: 175484534 , 18

In [0]:
#extract data from comma delimited columns
#drop unnecessary columns
#fill/Change missing values to Null
#remove erroneous data, sensor data has a quality rating in which we can through out data of bad quality and erroneous data collection
def weather_transformation(df):  
  return (
    df
       #WND - Wind direction and speed, Create substring columns delimited by ","
      .withColumn("WND_temp", f.substring_index("WND", ",", -2))\
      .withColumn("WND_SPD", f.substring_index("WND_temp", ",", 1))\
      .withColumn("WND_SPD_QUAL", f.substring_index("WND_temp", ",", -1))\
      .withColumn("WND_SPD_QUAL", f.when((f.col("WND_SPD_QUAL") == "3") | (f.col("WND_SPD_QUAL") == "7") , "999").otherwise(f.col("WND_SPD_QUAL")))\
      .withColumn("WND_SPD", f.when((f.col("WND_SPD") == "") | (f.col("WND_SPD") == "9999") | (f.col("WND_SPD_QUAL") == "999"), None).otherwise(f.col("WND_SPD")))\
      .drop("WND_temp","WND", "WND_SPD_QUAL")\
      #VISIBILITY-OBSERVATION, VIS - Create substring columns delimited by ","
      .withColumn("VIS_temp", f.substring_index("VIS", ",", 2))\
      .withColumn("VIS_DIST", f.substring_index("VIS_temp", ",", 1))\
      .withColumn("VIS_DIST_QUAL", f.substring_index("VIS_temp", ",", -1))\
      .withColumn("VIS_DIST_QUAL", f.when((f.col("VIS_DIST_QUAL") == "3") | (f.col("VIS_DIST_QUAL") == "7"), "999").otherwise(f.col("VIS_DIST_QUAL")))\
      .withColumn("VIS_DIST", f.when((f.col("VIS_DIST") == "") | (f.col("VIS_DIST") == "999999") | (f.col("VIS_DIST_QUAL") == "999"), None).otherwise(f.col("VIS_DIST")))\
      .drop("VIS_temp", "VIS_DIST_QUAL", "VIS")\
      #Temperature, TMP - Create substring columns delimited by ","
      .withColumn("TMP_TEMP", f.substring_index("TMP", ",", 1))\
      .withColumn("TMP_TEMP_QUAL", f.substring_index("TMP", ",", -1))\
      .withColumn("TMP_TEMP_QUAL", f.when((f.col("TMP_TEMP_QUAL") == "3") | (f.col("TMP_TEMP_QUAL") == "7"), "999").otherwise(f.col("TMP_TEMP_QUAL")))\
      .withColumn("TMP_TEMP", f.when((f.col("TMP_TEMP") == "") | (f.col("TMP_TEMP") == "+9999") | (f.col("TMP_TEMP_QUAL") == "999"), None).otherwise(f.col("TMP_TEMP")))\
      .drop("TMP_TEMP_QUAL", "TMP")\
      #Dew Point DEW - Create substring columns delimited by ","
      .withColumn("DEW_TEMP", f.substring_index("DEW", ",", 1))\
      .withColumn("DEW_TEMP_QUAL", f.substring_index("DEW", ",", -1))\
      .withColumn("DEW_TEMP_QUAL", f.when((f.col("DEW_TEMP_QUAL") == "3") | (f.col("DEW_TEMP_QUAL") == "7"), "999").otherwise(f.col("DEW_TEMP_QUAL")))\
      .withColumn("DEW_TEMP", f.when((f.col("DEW_TEMP") == "") | (f.col("DEW_TEMP") == "+9999") | (f.col("DEW_TEMP_QUAL") == "999"), None).otherwise(f.col("DEW_TEMP")))\
      .drop("DEW_TEMP_QUAL", "DEW")\
      #ATMOSPHERIC-PRESSURE-OBSERVATION, SLP - Create substring columns delimited by ","
      .withColumn("SLP_PRESSURE", f.substring_index("SLP", ",", 1))\
      .withColumn("SLP_PRESSURE_QUAL", f.substring_index("SLP", ",", -1))\
      .withColumn("SLP_PRESSURE_QUAL", f.when((f.col("SLP_PRESSURE_QUAL") == "3") | (f.col("SLP_PRESSURE_QUAL") == "7"), "999").otherwise(f.col("SLP_PRESSURE_QUAL")))\
      .withColumn("SLP_PRESSURE", f.when((f.col("SLP_PRESSURE") == "") | (f.col("SLP_PRESSURE") == "99999") | (f.col("SLP_PRESSURE_QUAL") == "999"), None).otherwise(f.col("SLP_PRESSURE")))\
      .drop("SLP_PRESSURE_QUAL", "SLP" )\
      #LIQUID-PRECIPITATION occurrence identifier, AA1 - Create substring columns delimited by ","
      .withColumn("AA1_temp", f.substring_index("AA1", ",", -3))\
      .withColumn("PRECIPITATION", f.substring_index("AA1_temp", ",", 1))\
      .withColumn("PRECIPITATION_QUAL", f.substring_index("AA1_temp", ",", -1))\
      .withColumn("PRECIPITATION_QUAL", f.when((f.col("PRECIPITATION_QUAL") == "3") | (f.col("PRECIPITATION_QUAL") == "7"), "999").otherwise(f.col("PRECIPITATION_QUAL")))\
      .withColumn("PRECIPITATION", f.when((f.col("PRECIPITATION") == "") | (f.col("PRECIPITATION") == "9999") | (f.col("PRECIPITATION_QUAL") == "999"), None).otherwise(f.col("PRECIPITATION")))\
      .drop("AA1_temp", "AA1", "PRECIPITATION_QUAL")\
      #SNOW-DEPTH identifier, AJ1 - Create substring columns delimited by ","
      .withColumn("AJ1_temp", f.substring_index("AJ1", ",", 3))\
      .withColumn("SNOW", f.substring_index("AJ1_temp", ",", 1))\
      .withColumn("SNOW_QUAL", f.substring_index("AJ1_temp", ",", -1))\
      .withColumn("SNOW_QUAL", f.when((f.col("SNOW_QUAL") == "3") | (f.col("SNOW_QUAL") == "7"), "999").otherwise(f.col("SNOW_QUAL")))\
      .withColumn("SNOW", f.when((f.col("SNOW") == "") | (f.col("SNOW") == "9999") | (f.col("SNOW_QUAL") == "999"), None).otherwise(f.col("SNOW")))\
      .drop("AJ1_temp", "AJ1", "SNOW_QUAL")\
      #PRESENT-WEATHER-OBSERVATION automated occurrence identifier for ASOS/AWOS data, AT1 - Create substring columns delimited by ","
      .withColumn("AT1_temp", f.substring_index("AT1", ",", -3))\
      .withColumn("WEATHER_OBSERVATION", f.substring_index("AT1_temp", ",", 1))\
      .withColumn("WEATHER_OBSERVATION_QUAL", f.substring_index("AT1_temp", ",", -1))\
      .withColumn("WEATHER_OBSERVATION_QUAL", f.when((f.col("WEATHER_OBSERVATION_QUAL") == "3") | (f.col("WEATHER_OBSERVATION_QUAL") == "7"), "999").otherwise(f.col("WEATHER_OBSERVATION_QUAL")))\
      .withColumn("WEATHER_OBSERVATION", f.when((f.col("WEATHER_OBSERVATION") == "") | (f.col("WEATHER_OBSERVATION_QUAL") == "999"), None).otherwise(f.col("WEATHER_OBSERVATION")))\
      .drop("AT1", "AT1_temp", "WEATHER_OBSERVATION_QUAL")\
      #SKY-COVER-LAYER, GA1 - Create substring columns delimited by ","
      .withColumn("GA1_temp", f.substring_index("GA1", ",", 4))\
      .withColumn("GA1_temp2", f.substring_index("GA1_temp", ",", 2))\
      .withColumn("GA1_temp3", f.substring_index("GA1_temp", ",", -2))\
      .withColumn("CLOUD_COVERAGE", f.substring_index("GA1_temp2", ",", 1))\
      .withColumn("CLOUD_COVERAGE_QUAL", f.substring_index("GA1_temp2", ",", -1))\
      .withColumn("CLOUD_COVERAGE_QUAL", f.when((f.col("CLOUD_COVERAGE_QUAL") == "3") | (f.col("CLOUD_COVERAGE_QUAL") == "7"), "999").otherwise(f.col("CLOUD_COVERAGE_QUAL")))\
      .withColumn("CLOUD_COVERAGE", f.when((f.col("CLOUD_COVERAGE") == "") | (f.col("CLOUD_COVERAGE") == "99") | (f.col("CLOUD_COVERAGE") == "9") | (f.col("CLOUD_COVERAGE") == "10") | (f.col("CLOUD_COVERAGE_QUAL") == "999"), None).otherwise(f.col("CLOUD_COVERAGE")))\
      .drop("GA1", "GA1_temp", "GA1_temp2", "CLOUD_COVERAGE_QUAL")\
      #SKY-COVER-LAYER, GA1 - Create substring columns delimited by ","
      .withColumn("CLOUD_BASE_HEIGHT", f.substring_index("GA1_temp3", ",", 1))\
      .withColumn("CLOUD_BASE_HEIGHT_QUAL", f.substring_index("GA1_temp3", ",", -1))\
      .withColumn("CLOUD_BASE_HEIGHT_QUAL", f.when((f.col("CLOUD_BASE_HEIGHT_QUAL") == "3") | (f.col("CLOUD_BASE_HEIGHT_QUAL") == "7"), "999").otherwise(f.col("CLOUD_BASE_HEIGHT_QUAL")))\
      .withColumn("CLOUD_BASE_HEIGHT", f.when((f.col("CLOUD_BASE_HEIGHT") == "") | (f.col("CLOUD_BASE_HEIGHT") == "+99999") | (f.col("CLOUD_BASE_HEIGHT_QUAL") == "999"), None).otherwise(f.col("CLOUD_BASE_HEIGHT")))\
      .drop("GA1_temp3", "CLOUD_BASE_HEIGHT_QUAL")\
      #Additional data section - IA1 - Create substring columns delimited by ","
      .withColumn("GROUND_SURFACE", f.substring_index("IA1", ",", 1))\
      .withColumn("GROUND_SURFACE_QUAL", f.substring_index("IA1", ",", -1))\
      .withColumn("GROUND_SURFACE_QUAL", f.when(f.col("GROUND_SURFACE_QUAL") == "3", "999").otherwise(f.col("GROUND_SURFACE_QUAL")))\
      .withColumn("GROUND_SURFACE", f.when((f.col("GROUND_SURFACE") == "") | (f.col("GROUND_SURFACE") == "99") | (f.col("GROUND_SURFACE_QUAL") == "999"), None).otherwise(f.col("GROUND_SURFACE")))\
      .drop("IA1", "GROUND_SURFACE_QUAL" )\
      #ATMOSPHERIC-PRESSURE-OBSERVATION identifier, MA1 - Create substring columns delimited by ","
      .withColumn("MA1_temp", f.substring_index("MA1", ",", 2))\
      .withColumn("ALTIMETER_SET", f.substring_index("MA1_temp", ",", 1))\
      .withColumn("ALTIMETER_SET_QUAL", f.substring_index("MA1_temp", ",", -1))\
      .withColumn("ALTIMETER_SET_QUAL", f.when((f.col("ALTIMETER_SET_QUAL") == "3") | (f.col("ALTIMETER_SET_QUAL") == "7"), "999").otherwise(f.col("ALTIMETER_SET_QUAL")))\
      .withColumn("ALTIMETER_SET", f.when((f.col("ALTIMETER_SET") == "") | (f.col("ALTIMETER_SET") == "99999") | (f.col("ALTIMETER_SET_QUAL") == "999"), None).otherwise(f.col("ALTIMETER_SET")))\
      .drop("MA1", "MA1_temp", "ALTIMETER_SET_QUAL")
  )
weather = weather_transformation(weather)
print("Rows, Columns: ", weather.count(), "," ,len(weather.columns))

Rows, Columns: 175484534 , 19

In [0]:
#drop missing values
weather = weather.drop(*missing_cols)
print("weather Rows, Columns: ", weather.count(), "," ,len(weather.columns))

weather Rows, Columns: 175484534 , 19

## Joins

In [0]:
#Station and Weather Join
st_id = stations.withColumn('id',f.concat(f.col('usaf'),f.col('wban')))
weather = weather.withColumn('cal_date', f.col('date').cast(DateType())).withColumn('time', f.concat(f.hour(f.col('date')),f.minute(f.col('date'))).cast(IntegerType()))
w_s = weather.join(st_id, weather.STATION==st_id.id, 'inner')
print("Rows, Columns: ", w_s.count(), "," ,len(w_s.columns))

Rows, Columns: 171541168 , 33

In [0]:
ws_final = w_s.join(codes.select('_c4','_c5'), w_s.call == codes._c5,'left')
print("Rows, Columns: ", ws_final.count(), "," ,len(ws_final.columns))

Rows, Columns: 171541168 , 35

In [0]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- WND_SPD: string (nullable = true)
-- VIS_DIST: string (nullable = true)
-- TMP_TEMP: string (nullable = true)
-- DEW_TEMP: string (nullable = true)
-- SLP_PRESSURE: string (nullable = true)
-- PRECIPITATION: string (nullable = true)
-- SNOW: string (nullable = true)
-- WEATHER_OBSERVATION: string (nullable = true)
-- CLOUD_COVERAGE: string (nullable = true)
-- CLOUD_BASE_HEIGHT: string (nullable = true)
-- GROUND_SURFACE: string (nullable = true)
-- ALTIMETER_SET: string (nullable = true)
-- cal_date: date (nullable = true)
-- time: integer (nullable = true)

In [0]:
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)

In [0]:
#Join Airlines to Station and Weather
airlines = airlines.withColumn('time',f.col('CRS_DEP_TIME').cast(IntegerType()))
ws_final = ws_final.withColumn('cal_date', ws_final['date'].cast(DateType()))
airlines.registerTempTable("airline")
ws_final.registerTempTable("weather")

# Join airlines with weather using the most recent weather station data at least two hours prior to planned departure time (two left joins)

aw_join = sqlContext.sql("SELECT * FROM (SELECT a.YEAR, a.MONTH, a.DAY_OF_WEEK, a.FL_DATE, a.OP_CARRIER, a.ORIGIN, a.ORIGIN_CITY_NAME, a.DEST, a.DEST_CITY_NAME, a.CRS_DEP_TIME, a.DEP_TIME, a.DEP_DEL15, a.DEP_TIME_BLK, a.TAXI_OUT, a.WHEELS_OFF, a.WHEELS_ON, a.TAXI_IN, a.CRS_ARR_TIME, a.DIVERTED, a.CRS_ELAPSED_TIME, a.DISTANCE, MAX(w.DATE) as w_date FROM airline a LEFT JOIN weather w ON a.ORIGIN == w._c4 AND w.cal_date == a.FL_DATE AND a.time - 200 >= w.time GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) m LEFT JOIN weather w ON m.w_date == w.DATE AND m.ORIGIN == w._c4 AND m.FL_DATE == w.cal_date")

print("Rows, Columns: ", aw_join.count(), "," ,len(aw_join.columns))

Rows, Columns: 31749281 , 57

In [0]:
final_fields_to_drop = ('REPORT_TYPE','WEATHER_OBSERVATION','GROUND_SURFACE','cal_date','_c4','_c5','country','call', 'id', 'state', 'name', 'usaf', 'wban', 'end', 'DATE', 'lon', 'elev', 'begin', 'STATION', 'ORIGIN_CITY_NAME', 'lat', 'CALL_SIGN', 'FL_Date', 'Quarter', 'DEST_CITY_NAME', 'DIVERTED','w_date','time', 'LATITUDE', 'LONGITUDE', 'DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'YEAR')

final_air_weather_df = aw_join.drop(*final_fields_to_drop)

## Polishing

##### Update Schema

In [0]:
print("Rows, Columns: ", final_air_weather_df.count(), "," ,len(final_air_weather_df.columns))

Rows, Columns: 31749281 , 21

In [0]:
final_air_weather_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- ORIGIN: string (nullable = true)
-- DEST: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- WND_SPD: string (nullable = true)
-- VIS_DIST: string (nullable = true)
-- TMP_TEMP: string (nullable = true)
-- DEW_TEMP: string (nullable = true)
-- SLP_PRESSURE: string (nullable = true)
-- PRECIPITATION: string (nullable = true)
-- SNOW: string (nullable = true)
-- CLOUD_COVERAGE: string (nullable = true)
-- CLOUD_BASE_HEIGHT: string (nullable = true)
-- ALTIMETER_SET: string (nullable = true)

##### Missing Values
Missing values need to be filled in.

In [0]:
#drop rows with no label for DEP_DEL15
final_air_weather_df = final_air_weather_df.na.drop(subset=["DEP_DEL15"])

In [0]:
#df with missing values
missing_df = count_missings(final_air_weather_df, sort=True)

# grab the columns that have missing values
missing_cols = missing_df[missing_df['count'] > 1].index.values
missing_cols

Out[27]: array(['SNOW', 'CLOUD_BASE_HEIGHT', 'PRECIPITATION', 'SLP_PRESSURE',
 'CLOUD_COVERAGE', 'DEW_TEMP', 'WND_SPD', 'TMP_TEMP', 'VIS_DIST',
 'ALTIMETER_SET', 'CRS_ELAPSED_TIME'], dtype=object)

In [0]:
from pyspark.sql.functions import isnan, when, count, col

display(final_air_weather_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final_air_weather_df.columns]))

MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,WND_SPD,VIS_DIST,TMP_TEMP,DEW_TEMP,SLP_PRESSURE,PRECIPITATION,SNOW,CLOUD_COVERAGE,CLOUD_BASE_HEIGHT,ALTIMETER_SET
0,0,0,0,0,0,0,0,0,31,0,269257,258384,263112,269661,567582,812092,31137267,560193,6211631,256617


In [0]:
display(final_air_weather_df.describe('CRS_ELAPSED_TIME', 'WND_SPD', 'VIS_DIST', 'TMP_TEMP', 'DEW_TEMP', 'SLP_PRESSURE', 'PRECIPITATION', 'CLOUD_COVERAGE', 'CLOUD_BASE_HEIGHT', 'ALTIMETER_SET'))

summary,CRS_ELAPSED_TIME,WND_SPD,VIS_DIST,TMP_TEMP,DEW_TEMP,SLP_PRESSURE,PRECIPITATION,CLOUD_COVERAGE,CLOUD_BASE_HEIGHT,ALTIMETER_SET
count,31271903,31002677,31013550,31008822,31002273,30704352,30459842,30711741,25060303,31015317
mean,143.44285168702396,34.67812482773665,14974.767891615116,153.53210150969295,87.06152716608875,10167.472085618352,1.0072120203381225,3.376440495509519,2428.0734019856027,10172.079096370351
stddev,74.8817510775698,23.51477572566233,3184.6082595951434,101.1434514019472,103.00498206275725,66.80345815384251,9.025382472406013,2.70745375296263,2528.6480369958076,63.61720542848712
min,-99.0,0000,000000,+0000,+0000,09603,0000,00,+00000,09560
max,813.0,0618,160000,-0439,-0422,10598,2557,09,+30023,10826


`SNOW` has 98% empty values therefore it will be removed from our dataset.

__Replacing null values with Mean and Median__

- __MEDIAN__
  - Empty values in the following columns will be replace by __Median__ if the `mean` is to `min` and `max`.
  - Empty values of discrete variables will be set to their __Median__.
  - `WND_SPD, VIS_DIST, PRECIPITATION, CLOUD_COVERAGE, CLOUD_BASE_HEIGHT, ALTIMETER_SET,`
- __MEAN__
  - Rest of the empty values will be replaced be __Mean__ are they likely do not have a skewed distribution.
  - `TMP_TEMP, DEW_TEMP, SLP_PRESSURE, CRS_ELAPSED_TIME`

In [0]:
#removing 'SNOW' as 98% of its data is missing
final_air_weather_df = final_air_weather_df.drop("SNOW")

#cast columns to INT
to_int = ['WND_SPD', 'VIS_DIST', 'TMP_TEMP', 'DEW_TEMP', 'SLP_PRESSURE', 'PRECIPITATION', 'CLOUD_COVERAGE', 'CLOUD_BASE_HEIGHT', 'ALTIMETER_SET']
 
for c in to_int:
  final_air_weather_df = final_air_weather_df.withColumn(c, final_air_weather_df[c].cast("int"))
  
replace_by_mean = ['TMP_TEMP', 'DEW_TEMP', 'SLP_PRESSURE', 'CRS_ELAPSED_TIME']

replace_by_median = ['WND_SPD', 'VIS_DIST', 'PRECIPITATION', 'CLOUD_COVERAGE', 'CLOUD_BASE_HEIGHT', 'ALTIMETER_SET']

#replace null values with mean value
for c in replace_by_mean:
  c_mean = final_air_weather_df.agg({c: 'mean'}).collect()[0][0]
  final_air_weather_df = final_air_weather_df.na.fill({c: c_mean})
  
#replace null values with median value
for c in replace_by_median:
  c_median = final_air_weather_df.approxQuantile(c, [0.5],0.1)[0]
  final_air_weather_df = final_air_weather_df.na.fill({c: c_median})

In [0]:
display(final_air_weather_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final_air_weather_df.columns]))

MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,WND_SPD,VIS_DIST,TMP_TEMP,DEW_TEMP,SLP_PRESSURE,PRECIPITATION,CLOUD_COVERAGE,CLOUD_BASE_HEIGHT,ALTIMETER_SET
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# check shape before output
print("Rows, Columns: ", final_air_weather_df.count(), "," ,len(final_air_weather_df.columns))

Rows, Columns: 31271934 , 20

## Output

`Read parquet file:`
`final_air_weather = spark.read.option("header", "true").parquet(f"dbfs:/tmp/out/final_air_weather.parquet")`

In [0]:
final_air_weather_df.write.mode("overwrite").parquet("/tmp/out/final_air_weather.parquet") 